##### Copyright 2024 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.0 - Multimodal live API tool use with websockets

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/gemini-2/websockets/live_api_tool_use.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook provides examples of how to use tools with the Multimodal Live API with the Gemini 2.0 models. The API provides Google Search, Code Execution and Function Calling.

This tutorial assumes you are familiar with the Live API, as described in the [Live API starter tutorial](live_api_starter.ipynb).

Note: This version of the tutorial uses websockets directly. The [SDK version of this tutorial](../live_api_tool_use.ipynb) is a bit simpler because the SDK handles some of the details for you.

## Set up

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../../quickstarts/Authentication.ipynb) for an example.

In [9]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

Install the `websockets` package:

In [6]:
!pip install -q websockets

import the necessary modules

In [10]:
import asyncio
import base64
import contextlib
import os
import json
import wave

from IPython import display

from websockets.asyncio.client import connect

In [11]:
uri = f"wss://generativelanguage.googleapis.com/ws/google.ai.generativelanguage.v1alpha.GenerativeService.BidiGenerateContent?key={GOOGLE_API_KEY}"

model = "models/gemini-2.0-flash-exp"

Define a context manager to convert streamed PCM data into a wave file that can be played directly using an IPython audio widget.

In [14]:
@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

And define a custom logger so you can toggle extra information, like the in-flight requests and responses.

In [15]:
import logging

logger = logging.getLogger("Live")
logger.setLevel("INFO")

These helpers handle the websocket connection and prompt transmission (`run` and `send`), server handshake (`setup`) and process server responses (`handle_server_content`, `handle_tool_call`).

In [16]:
async def setup(ws, modality, tools):
  setup = {
      "setup": {
          "model": model,
          "tools": tools,
          "generation_config": {
              "response_modalities": [modality]
              }}}
  await ws.send(json.dumps(setup))
  setup_response = json.loads(await ws.recv())
  logger.debug(setup_response)

async def send(ws, prompt):
  msg = {
    "client_content": {
      "turns": [{"role": "user", "parts": [{"text": prompt}]}],
      "turn_complete": True,
    }
  }
  print(">>> ", msg)
  await ws.send(json.dumps(msg))


def handle_server_content(wf, server_content):
  audio = False
  model_turn = server_content.pop("modelTurn", None)
  if model_turn:
    text = model_turn["parts"][0].pop("text", None)
    if text is not None:
      print(text)

    inline_data = model_turn['parts'][0].pop('inlineData', None)
    if inline_data is not None:
      print('.', end='')
      b64data = inline_data['data']
      pcm_data = base64.b64decode(b64data)
      wf.writeframes(pcm_data)
      audio = True

  turn_complete = server_content.pop('turnComplete', None)
  return turn_complete, audio


async def handle_tool_call(ws, tool_call):
  print("    ", tool_call)
  for fc in tool_call['functionCalls']:

    msg = {
      'tool_response': {
          'function_responses': [{
              'id': fc['id'],
              'name': fc['name'],
              'response':{'result': {'string_value': 'ok'}}
          }]
        }
    }
    print('>>> ', msg)
    await ws.send(json.dumps(msg))



async def run(prompt, modality='TEXT', tools=None):
  if tools is None:
    tools=[]

  async with (
      connect(uri, additional_headers={"Content-Type": "application/json"}) as ws,
  ):
    await setup(ws, modality, tools)
    await send(ws, prompt)

    audio = False
    filename = 'audio.wav'
    with wave_file(filename) as wf:
      async for raw_response in ws:
        response = json.loads(raw_response.decode())
        logger.debug(str(response)[:150])

        server_content = response.pop("serverContent", None)
        if server_content is not None:
          turn_complete, a = handle_server_content(wf, server_content)
          audio = audio or a
          if turn_complete:
            print()
            print('Turn complete')
            break

        tool_call = response.pop('toolCall', None)
        if tool_call is not None:
          await handle_tool_call(ws, tool_call)

  if audio:
    display.display(display.Audio(filename, autoplay=True))

Run a test prompt to ensure everything is set up.

In [17]:
await run(prompt='Hello?')

>>>  {'client_content': {'turns': [{'role': 'user', 'parts': [{'text': 'Hello?'}]}], 'turn_complete': True}}
Hello
 there! How can I help you today?


Turn complete


## Simple function call

Define some stub functions to use in a function calling example.

In [13]:
turn_on_the_lights_schema = {'name': 'turn_on_the_lights'}
turn_off_the_lights_schema = {'name': 'turn_off_the_lights'}

Send the function declarations as part of the `tools` (in the generation config).

In [18]:
prompt = "Turn on the lights"

tools = [
    {'function_declarations': [turn_on_the_lights_schema, turn_off_the_lights_schema]}
]

await run(prompt, tools=tools)

>>>  {'client_content': {'turns': [{'role': 'user', 'parts': [{'text': 'Turn on the lights'}]}], 'turn_complete': True}}
     {'functionCalls': [{'name': 'turn_on_the_lights', 'args': {}, 'id': 'function-call-12001301744900464704'}]}
>>>  {'tool_response': {'function_responses': [{'id': 'function-call-12001301744900464704', 'name': 'turn_on_the_lights', 'response': {'result': {'string_value': 'ok'}}}]}}
OK
. The lights are now on.


Turn complete


Try the same thing again, but using audio-out this time.

In [19]:
prompt = "Turn on the lights"

tools = [
    {'function_declarations': [turn_on_the_lights_schema, turn_off_the_lights_schema]}
]

await run(prompt, tools=tools, modality = "AUDIO")

>>>  {'client_content': {'turns': [{'role': 'user', 'parts': [{'text': 'Turn on the lights'}]}], 'turn_complete': True}}
     {'functionCalls': [{'name': 'turn_on_the_lights', 'args': {}, 'id': 'function-call-15210722669530560737'}]}
>>>  {'tool_response': {'function_responses': [{'id': 'function-call-15210722669530560737', 'name': 'turn_on_the_lights', 'response': {'result': {'string_value': 'ok'}}}]}}
..........
Turn complete


## Code execution

The API can generate and execute code during the conversation too.

In [20]:
prompt="What is the largest prime palindrome under 100000"

tools = [
    {'code_execution': {}}
]

await run(prompt, tools=tools, modality='AUDIO')

>>>  {'client_content': {'turns': [{'role': 'user', 'parts': [{'text': 'What is the largest prime palindrome under 100000'}]}], 'turn_complete': True}}
......................................................
Turn complete


## Google search

A `google_search` tool is also available for use during live conversations.

In [21]:
prompt="Can you use google search tell me about the largest earthquake in california the week of Dec 5 2024?"

tools = [
   {'google_search': {}}
]

await run(prompt, tools=tools, modality='TEXT')

>>>  {'client_content': {'turns': [{'role': 'user', 'parts': [{'text': 'Can you use google search tell me about the largest earthquake in california the week of Dec 5 2024?'}]}], 'turn_complete': True}}


The
 largest earthquake in California during the week of December 5, 202
4, was a magnitude 7.0 that occurred offshore of Cape Mendoc
ino on December 5, 2024, at 10:44 AM PT. The earthquake was located approximately 60 miles (
100km) southwest of Ferndale, in Northern California. The earthquake prompted a brief tsunami warning for coastal areas from Santa Cruz into Oregon, which was
 canceled shortly after.

Here's a summary of the key details:

*   **Magnitude:** 7.0
*   **Date:** December 5, 2024
*   **Time:** 1
0:44 AM PT
*   **Location:** Approximately 60 miles (100km) southwest of Ferndale, California, offshore of Cape Mendocino
*   **Tsunami Warning:** A tsunami warning was
 issued but later canceled.
*   **Aftershocks:** There were many aftershocks following the main earthquake

Try the same again, with audio.

In [22]:
prompt="Can you use google search tell me about the largest earthquake in california the week of Dec 5 2024?"

tools = [
   {'google_search': {}}
]

await run(prompt, tools=tools, modality='AUDIO')

>>>  {'client_content': {'turns': [{'role': 'user', 'parts': [{'text': 'Can you use google search tell me about the largest earthquake in california the week of Dec 5 2024?'}]}], 'turn_complete': True}}
...............................................................................................................................................................................................
Turn complete


## Compositional Function Calling

TODO: it seems to work but you can't see it.

In [ ]:
prompt = """
  Hey, can you write run some python code to turn on the lights, wait 10s and then turn off the lights?
  """

tools = [
    {'code_execution': {}},
    {'function_declarations': [turn_on_the_lights_schema, turn_off_the_lights_schema]}
]

await run(prompt, tools=tools, modality="AUDIO")

## Multi-tool


In [ ]:
prompt = """
  Hey, I need you to do three things for me.

  1. Turn on the lights
  2. Then compute the largest prime plaindrome under 100000.
  3. Then use google search to lookup unformation about the largest earthquake in california the week of Dec 5 2024?

  Thanks!
  """

tools = [
    {'google_search': {}},
    {'code_execution': {}},
    {'function_declarations': [turn_on_the_lights_schema, turn_off_the_lights_schema]}
]

await run(prompt, tools=tools, modality="AUDIO")

## Next steps

<a name="next_steps"></a>

This tutorial just shows basic usage of the Live API, using the Python GenAI SDK.

- If you aren't looking for code, and just want to try multimedia streaming use [Live API in Google AI Studio](https://aistudio.google.com/app/live).
- If you want to see how to setup streaming interruptible audio and video using the Live API and the SDK see the [Audio and Video input Tutorial](../../gemini-2/live_api_starter.py).
- There is a [Streaming audio in Colab example](../../gemini-2/websockets/live_api_streaming_in_colab.ipynb), but this is more of a **demo**, it's **not optimized for readability**.
- Other nice Gemini 2.0 examples can also be found in the [Cookbook](https://github.com/google-gemini/cookbook/blob/main/gemini-2/).
